In [8]:
# from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack.components.embedders import SentenceTransformersDocumentEmbedder

# 建立嵌入器，並指定要嵌入的元數據欄位
embedder = SentenceTransformersDocumentEmbedder(
    meta_fields_to_embed=["url"]
)

In [9]:
from haystack import Pipeline
from haystack.components.preprocessors import DocumentCleaner, DocumentSplitter
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack.components.writers import DocumentWriter
from haystack.document_stores.types import DuplicatePolicy
from haystack.utils import ComponentDevice

若要使用中文

In [10]:
# 設置維基百科語言為中文
wikipedia.set_lang("zh")

In [11]:
# 定義一個輔助函數：建立索引管道
def create_indexing_pipeline(
    document_store, metadata_fields_to_embed=None
):
    # 建立文件清理器
    document_cleaner = DocumentCleaner()
    # 建立文件分割器，按句子分割
    document_splitter = DocumentSplitter(
        split_by="sentence", split_length=2
    )
    # 建立文件嵌入器，並指定要嵌入的元數據欄位
    document_embedder = SentenceTransformersDocumentEmbedder(
        model="thenlper/gte-large",
        meta_fields_to_embed=metadata_fields_to_embed
    )
    # 建立文件寫入器，使用覆寫策略
    document_writer = DocumentWriter(
        document_store=document_store,
        policy=DuplicatePolicy.OVERWRITE
    )
    # 建立管道
    indexing_pipeline = Pipeline()
    # 添加文件清理器
    indexing_pipeline.add_component(
        "cleaner", document_cleaner
    )
    # 添加文件分割器
    indexing_pipeline.add_component(
        "splitter", document_splitter
    )
    # 添加文件嵌入器
    indexing_pipeline.add_component(
        "embedder", document_embedder
    )
    # 添加文件寫入器
    indexing_pipeline.add_component(
        "writer", document_writer
    )

    # 連接管道組件
    indexing_pipeline.connect("cleaner", "splitter")
    indexing_pipeline.connect("splitter", "embedder")
    indexing_pipeline.connect("embedder", "writer")
    # 返回配置好的管道
    return indexing_pipeline

In [12]:
import wikipedia
from haystack import Document
from haystack.document_stores.in_memory import InMemoryDocumentStore

# 定義需要檢索的樂隊名稱 `披頭四、怪人合唱團（治療樂隊）`
# some_bands = """The Beatles,The Cure""".split(",")
some_bands = """五月天,信樂團""".split(",")

raw_docs = []

# 從維基百科取得每個樂隊的頁面，並建立文件
for title in some_bands:
    page = wikipedia.page(
        title=title, auto_suggest=False
    )
    doc = Document(
        content=page.content,
        meta={
            "title": page.title,
            "url": page.url
        }
    )
    # 將建立的文件添加到列表中
    raw_docs.append(doc)

# 建立記憶體文件儲存
document_store = InMemoryDocumentStore(
    embedding_similarity_function="cosine"
)
document_store_with_embedded_metadata = InMemoryDocumentStore(
    embedding_similarity_function="cosine"
)

# 建立僅索引內容的管道
indexing_pipeline = create_indexing_pipeline(
    document_store=document_store
)

# 建立索引內容和元數據的管道
indexing_with_metadata_pipeline = create_indexing_pipeline(
    document_store=document_store_with_embedded_metadata,
    metadata_fields_to_embed=["title"]
)

# 執行管道以索引文件
indexing_pipeline.run({
    "cleaner": {"documents": raw_docs}
})
indexing_with_metadata_pipeline.run({
    "cleaner": {"documents": raw_docs}
})

Batches: 100%|██████████| 1/1 [00:02<00:00,  2.17s/it]


{'writer': {'documents_written': 12}}

In [13]:
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever

# 建立檢索管道
retrieval_pipeline = Pipeline()
retrieval_pipeline.add_component(
    "text_embedder",
    SentenceTransformersTextEmbedder(model="thenlper/gte-large")
)
retrieval_pipeline.add_component(
    "retriever", InMemoryEmbeddingRetriever(
        document_store=document_store,
        scale_score=False,
        top_k=3
    )
)
retrieval_pipeline.add_component(
    "retriever_with_embeddings",
    InMemoryEmbeddingRetriever(
        document_store=document_store_with_embedded_metadata,
        scale_score=False, top_k=3
    )
)

# 連接檢索管道組件
retrieval_pipeline.connect(
    "text_embedder", "retriever"
)
retrieval_pipeline.connect(
    "text_embedder", "retriever_with_embeddings"
)

# 執行檢索並比較結果
result = retrieval_pipeline.run({
    "text_embedder": {"text": "Have the Beatles ever been to Bangor?"}
})
# 輸出結果一
print("Retriever Results:\n")
# 遍歷
for doc in result["retriever"]["documents"]:
    print(doc)
# 輸出結果二
print("Retriever with Embeddings Results:\n")
# 遍歷
for doc in result["retriever_with_embeddings"]["documents"]:
    print(doc)

Batches: 100%|██████████| 1/1 [00:00<00:00, 16.39it/s]

Retriever Results:

Document(id=50e230c396fddb360d4ef76f7c715285ad7824d8ca9d758237d67c71a51b83f6, content: 'E、徐佳瑩、梁靜茹、鄧紫棋、家家、艾怡良、魏如萱、黃韻玲 。
8月底，五月天對外發佈他們將閉關半年至一年，甚至中斷參與跨年演唱會的十三年紀錄，全心投入新專輯的籌備製作。 === 2016年：第九張錄音...', meta: {'title': '五月天', 'url': 'https://zh.wikipedia.org/wiki/%E4%BA%94%E6%9C%88%E5%A4%A9', 'source_id': '7c9844c6bb95b87986ea230d2b1de1e32fe0f010e2298411f48d13a300823299', 'page_number': 1}, score: 0.745767090764684)
Document(id=ae47234e93b13929ddd33b4dc2a5436316ff2f8dab5ebd8551d0a07ffdc57df3, content: 'N.A創造世界巡迴演唱會」最終場後，五月天開始籌備新專輯，短期內沒有大型巡迴演出。由於相信音樂旗下藝人梁靜茹合約轉出，臺北小巨蛋的演唱會無法如期舉行，討論後改由五月天領軍八組樂團，於2010年8月14...', meta: {'title': '五月天', 'url': 'https://zh.wikipedia.org/wiki/%E4%BA%94%E6%9C%88%E5%A4%A9', 'source_id': '7c9844c6bb95b87986ea230d2b1de1e32fe0f010e2298411f48d13a300823299', 'page_number': 1}, score: 0.7401567288991424)
Document(id=75efd8c6575f7168129e409aaf417e958166df9535a9e36be27282a7349b93b4, content: 'N.D」來自「D.', meta: {'title': '五月天', 'url': 'https://zh.wikipedia.org/w

優化輸出

In [14]:
# 格式化輸出
def format_document_output(documents, title):
    print(f"{title} Results:\n")
    for doc in documents:
        print(f"Document ID: {doc.id}")
        print(f"Content: {doc.content[:200]}...")  # 只顯示前200個字元
        print(f"Metadata: {doc.meta}")
        print(f"Score: {doc.score}\n")
        print("-" * 80)

# 輸出結果一
format_document_output(result["retriever"]["documents"], "Retriever")

# 輸出結果二
format_document_output(result["retriever_with_embeddings"]["documents"], "Retriever with Embeddings")

Retriever Results:

Document ID: 50e230c396fddb360d4ef76f7c715285ad7824d8ca9d758237d67c71a51b83f6
Content: E、徐佳瑩、梁靜茹、鄧紫棋、家家、艾怡良、魏如萱、黃韻玲 。
8月底，五月天對外發佈他們將閉關半年至一年，甚至中斷參與跨年演唱會的十三年紀錄，全心投入新專輯的籌備製作。 === 2016年：第九張錄音室專輯《自傳》 ===
2016年5月20日至6月1日，五月天在香港紅磡體育館連續舉辦了十場「Just Rock It!!! 就是」世界巡迴演唱會，打破了海外歌手在該場地演出場次最多的記錄。同日，公佈...
Metadata: {'title': '五月天', 'url': 'https://zh.wikipedia.org/wiki/%E4%BA%94%E6%9C%88%E5%A4%A9', 'source_id': '7c9844c6bb95b87986ea230d2b1de1e32fe0f010e2298411f48d13a300823299', 'page_number': 1}
Score: 0.745767090764684

--------------------------------------------------------------------------------
Document ID: ae47234e93b13929ddd33b4dc2a5436316ff2f8dab5ebd8551d0a07ffdc57df3
Content: N.A創造世界巡迴演唱會」最終場後，五月天開始籌備新專輯，短期內沒有大型巡迴演出。由於相信音樂旗下藝人梁靜茹合約轉出，臺北小巨蛋的演唱會無法如期舉行，討論後改由五月天領軍八組樂團，於2010年8月14日舉行「超犀利趴」演唱會。九個小時馬拉松式的演唱會，引領觀眾穿人字拖入場，亦開放在館內飲食，均造成話題，再創紀錄。 === 2011年：電影「追夢3DNA」與《第二人生》 ===
2011年5月20日...
Metadata: {'title': '五月天', 'url': 'https://zh.wikipedia.org/wiki/%E4%BA%94%E6%9C%88%E5%A4%A9', 's